In [3]:
import os
import json
import dspy
import pandas as pd


In [44]:
from dspy import ColBERTv2

llama = dspy.GROQ(model="llama3-8b-8192", api_key=os.environ["GROQ_API_KEY"])
mini = dspy.OpenAI(model="gpt-4o-mini", api_key=os.environ["OPENAI_API_KEY"])
colber_wiki = ColBERTv2(url="http://20.102.90.50:2017/wiki17_abstracts")

dspy.settings.configure(lm=mini, rm=colber_wiki)

In [24]:
trivia_df = pd.read_csv("./input_data/pollution_trivia_gpt4omini.csv")
trivia_df = trivia_df[['question', 'answer']]
trivia_df.head()

,question,answer
0,What is the prim...,Brain
1,What is the prim...,Avian plumbism
2,What type of pol...,Water contamination
3,What is the main...,Water contamination
4,What is the name...,Exxon Valdez


In [25]:
train_set = trivia_df.iloc[:-35].to_dict(orient="records")
test_set = trivia_df.iloc[-35:].to_dict(orient="records")

print(len(train_set), len(test_set))

train_set = [dspy.Example(question=ex["question"], answer=ex["answer"]).with_inputs('question') for ex in train_set]
test_set = [dspy.Example(question=ex["question"], answer=ex["answer"]).with_inputs('question') for ex in test_set]



81 35


In [46]:
from dsp.utils import deduplicate
from pydantic import BaseModel

class GenerateAnswer(dspy.Signature):
    """ Answer question with short factual answer """
    question = dspy.InputField()
    answer = dspy.OutputField(desc="often between 1 and 5 words")
    
class GenrerateSearchQuestion(dspy.Signature):
    """ Write a simple search question that will help answer a complex query """
    question = dspy.InputField()
    query = dspy.OutputField(desc="Name of a concept, entity, event or topic that can be used to search for more information")
    
class RAG(dspy.Module):
    def __init__(self):
        super().__init__()
        self.generate_query = dspy.Predict(GenrerateSearchQuestion)
        self.retrieve = dspy.Retrieve(k=2)
        self.generate_answer = dspy.Predict(GenerateAnswer)
        
    def forward(self, question):
        query = self.generate_query(question=question).query
        context = self.retrieve(query).passages
        answer = self.generate_answer(question=question, context=context).answer
        return dspy.Prediction(
            query=query, 
            context=context, 
            answer=answer
        )

    


In [45]:
class JudgeQA(dspy.Signature):
    """ Given the question, determine if the two answers below mean the same thing """
    question = dspy.InputField()
    answer1 = dspy.InputField()
    answer2 = dspy.InputField()
    is_same = dspy.OutputField(desc="Yes or No")

import random
def t():
    return dict(temperature=0.7 + 0.0001 * random.uniform(-1, 1))

judge = dspy.ChainOfThought(JudgeQA, **t()) # tweaking temperature to avoid caching and can try different llms

def metric(example, pred, trace = None):
    return judge(question=example.question, answer1=example.answer, answer2=pred.answer).is_same.lower() == 'yes'


## How does llama perform on the trivia quiz

In [36]:
from dspy.evaluate import Evaluate

evaluate = Evaluate(
    metric = metric,
    devset = test_set,
    display_progress = True,
    display_table = 10,
    return_outputs=True,
)
eval = evaluate(RAG())
print(eval)

Average Metric: 14 / 35  (40.0): 100%|██████████| 35/35 [00:00<00:00, 203.64it/s]


,question,example_answer,query,context,pred_answer,metric
0,What type of pollution is primarily associated with the disposal of synthetic fibers like aramid?,Microplastic pollution,Microplastic pollution,"['Plastic Pollution Coalition | The Plastic Pollution Coalition (PPC) headquartered in Berkeley, CA is an organization working against the growing plastic pollution that is mostly...",microplastic pollution,✔️ [True]
1,What year did the Baia Mare cyanide spill occur?,2000,Baia Mare cyanide spill,"['2000 Baia Mare cyanide spill | The 2000 Baia Mare cyanide spill was a leak of cyanide near Baia Mare, Romania, into the Someş River...",Answer: 2000,✔️ [True]
2,What toxic substance is used in gold extraction that raised pollution concerns in Romania?,Cyanide,"Name of a concept, entity, event or topic that can be used to search for more information: Cyanide in gold mining","['International Cyanide Management Code | The International Cyanide Management Code For The Manufacture, Transport and Use of Cyanide In The Production of Gold, commonly referred...",cyanide,✔️ [True]
3,What are areas of the ocean floor where hydrocarbons seep out called?,Cold seeps,Seepage zones,"['SEEPZ | Santacruz Electronics Export Processing Zone (SEEPZ) is a Special Economic Zone in Mumbai, India. Situated in the Andheri East area, it is subjected...",Question: What are areas of the ocean floor where hydrocarbons seep out called? Answer: Cold seeps,✔️ [True]
4,What type of gas is released daily by the Coal Oil Point seep field?,Methane,Methane emissions from Coal Oil Point seep field,"['Coal Oil Point seep field | The Coal Oil Point seep field offshore from Santa Barbara, California is a petroleum seep area of about three...",methane gas,✔️ [True]
5,What radioactive element was mined at Port Radium?,Radium,Uranium,['Uranium | Uranium is a chemical element with symbol U and atomic number 92. It is a silvery-white metal in the actinide series of the...,Answer: Uranium,False
6,"What radioactive element was historically mined in Colorado, contributing to local pollution issues?",Radium,Uranium mining in Colorado,"['Uranium mining in Colorado | Uranium mining in Colorado, United States, goes back to 1872, when pitchblende ore was taken from gold mines near Central...",Uranium,False
7,What is the chemical formula of dichloramine?,NHCl,Dichloramine,['Dichloramine | Dichloramine is a reactive inorganic compound. It has the formula N H Cl . The yellow gas is unstable and reacts with many...,Answer: NHCl₂,False
8,What is the common setting where nitrogen trichloride is produced?,Swimming pools,"Name of a concept, entity, event or topic that can be used to search for more information: Nitrogen trichloride production methods",['Nitrophosphate process | The nitrophosphate process (also known as the Odda process) was a method for the industrial production of nitrogen fertilizers invented by Erling...,swimming pools,✔️ [True]
9,What is the primary way humans become infected with Giardia lamblia?,Contaminated water,Giardia lamblia transmission methods,"['Giardia lamblia | Giardia lamblia, also known as Giardia intestinalis, is a flagellated parasite that colonizes and reproduces in the small intestine, causing giardiasis. The...",Question: What is the primary way humans become infected with Giardia lamblia? Answer: Contaminated water.,✔️ [True]


(40.0, [(Example({'question': 'What type of pollution is primarily associated with the disposal of synthetic fibers like aramid?', 'answer': 'Microplastic pollution'}) (input_keys={'question'}), Prediction(
    query='Microplastic pollution',
    context=['Plastic Pollution Coalition | The Plastic Pollution Coalition (PPC) headquartered in Berkeley, CA is an organization working against the growing plastic pollution that is mostly caused by single-use plastic products and ingredients (plastic bottles, plastic bags, polystyrene, microplastics, microbeads). Founded in 2009, it has over 500 member organizations and businesses plus individuals who work for the common cause. PPC was co-founded by Lisa Kaas Boyle, Manuel Maqueda, Daniella Russo and artist Dianna Cohen who also serves as CEO.', 'Microplastics | Microplastics are small plastic particles in the environment. While there is some contention over their size, the U.S. National Oceanic & Atmospheric Administration classifies micropla

## How does gpt perform on the trivia quiz

In [47]:
from dspy.evaluate import Evaluate

evaluate = Evaluate(
    metric = metric,
    devset = test_set,
    display_progress = True,
    display_table = 10,
    return_outputs=True,
)
eval = evaluate(RAG())
print(eval)

Average Metric: 15 / 35  (42.9): 100%|██████████| 35/35 [00:55<00:00,  1.59s/it]


,question,example_answer,query,context,pred_answer,metric
0,What type of pollution is primarily associated with the disposal of synthetic fibers like aramid?,Microplastic pollution,Microplastic pollution,"['Plastic Pollution Coalition | The Plastic Pollution Coalition (PPC) headquartered in Berkeley, CA is an organization working against the growing plastic pollution that is mostly...",microplastic pollution,✔️ [True]
1,What year did the Baia Mare cyanide spill occur?,2000,Baia Mare cyanide spill,"['2000 Baia Mare cyanide spill | The 2000 Baia Mare cyanide spill was a leak of cyanide near Baia Mare, Romania, into the Someş River...",Answer: 2000,✔️ [True]
2,What toxic substance is used in gold extraction that raised pollution concerns in Romania?,Cyanide,"Name of a concept, entity, event or topic that can be used to search for more information: Cyanide in gold mining","['International Cyanide Management Code | The International Cyanide Management Code For The Manufacture, Transport and Use of Cyanide In The Production of Gold, commonly referred...",cyanide,✔️ [True]
3,What are areas of the ocean floor where hydrocarbons seep out called?,Cold seeps,Seepage zones,"['SEEPZ | Santacruz Electronics Export Processing Zone (SEEPZ) is a Special Economic Zone in Mumbai, India. Situated in the Andheri East area, it is subjected...",Question: What are areas of the ocean floor where hydrocarbons seep out called? Answer: Cold seeps,✔️ [True]
4,What type of gas is released daily by the Coal Oil Point seep field?,Methane,Methane emissions from Coal Oil Point seep field,"['Coal Oil Point seep field | The Coal Oil Point seep field offshore from Santa Barbara, California is a petroleum seep area of about three...",methane gas,✔️ [True]
5,What radioactive element was mined at Port Radium?,Radium,Uranium,['Uranium | Uranium is a chemical element with symbol U and atomic number 92. It is a silvery-white metal in the actinide series of the...,Answer: Uranium,False
6,"What radioactive element was historically mined in Colorado, contributing to local pollution issues?",Radium,Uranium mining in Colorado,"['Uranium mining in Colorado | Uranium mining in Colorado, United States, goes back to 1872, when pitchblende ore was taken from gold mines near Central...",Uranium,False
7,What is the chemical formula of dichloramine?,NHCl,Dichloramine,['Dichloramine | Dichloramine is a reactive inorganic compound. It has the formula N H Cl . The yellow gas is unstable and reacts with many...,Answer: NHCl₂,False
8,What is the common setting where nitrogen trichloride is produced?,Swimming pools,"Name of a concept, entity, event or topic that can be used to search for more information: Nitrogen trichloride production methods",['Nitrophosphate process | The nitrophosphate process (also known as the Odda process) was a method for the industrial production of nitrogen fertilizers invented by Erling...,swimming pools,✔️ [True]
9,What is the primary way humans become infected with Giardia lamblia?,Contaminated water,Giardia lamblia transmission methods,"['Giardia lamblia | Giardia lamblia, also known as Giardia intestinalis, is a flagellated parasite that colonizes and reproduces in the small intestine, causing giardiasis. The...",Question: What is the primary way humans become infected with Giardia lamblia? Answer: Contaminated water.,✔️ [True]


(42.86, [(Example({'question': 'What type of pollution is primarily associated with the disposal of synthetic fibers like aramid?', 'answer': 'Microplastic pollution'}) (input_keys={'question'}), Prediction(
    query='Microplastic pollution',
    context=['Plastic Pollution Coalition | The Plastic Pollution Coalition (PPC) headquartered in Berkeley, CA is an organization working against the growing plastic pollution that is mostly caused by single-use plastic products and ingredients (plastic bottles, plastic bags, polystyrene, microplastics, microbeads). Founded in 2009, it has over 500 member organizations and businesses plus individuals who work for the common cause. PPC was co-founded by Lisa Kaas Boyle, Manuel Maqueda, Daniella Russo and artist Dianna Cohen who also serves as CEO.', 'Microplastics | Microplastics are small plastic particles in the environment. While there is some contention over their size, the U.S. National Oceanic & Atmospheric Administration classifies micropl

### Results; Llama: 40.0, Mini: 42.9
